##### Copyright 2023 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TF-NumPy 형식 승격

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/tf_numpy_type_promotion"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/guide/tf_numpy_type_promotion.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a>
</td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/guide/tf_numpy_type_promotion.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/guide/tf_numpy_type_promotion.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a></td>
</table>

## 개요

TensorFlow의 형식 승격에는 4가지 옵션이 있습니다.

- 기본적으로 TensorFlow는 혼합 형식 연산에서 형식을 승격하는 대신 오류를 발생시킵니다.
- `tf.numpy.experimental_enable_numpy_behavior()`를 실행하면 [NumPy 형식 승격 규칙](https://www.tensorflow.org/guide/tf_numpy#type_promotion)을 사용하도록 TensorFlow를 전환합니다.
- **이 문서**는 TensorFlow 2.15(또는 현재 `tf-nightly`)에서 사용할 수 있는 두 가지 새로운 옵션을 설명합니다.

In [ ]:
!pip install -q tf_nightly

**참고**: `experimental_enable_numpy_behavior`는 모든 TensorFlow의 작동을 변경합니다.

## 설치하기

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.experimental.numpy as tnp

print("Using TensorFlow version %s" % tf.__version__)

### 새로운 형식 승격 활성화하기

TF-Numpy에서 [JAX와 유사한 형식의 승격](https://jax.readthedocs.io/en/latest/type_promotion.html)을 사용하려면 TensorFlow에 대한 NumPy 작동을 활성화할 때 `'all'` 또는 `'safe'`를 dtype 변환 모드로 지정합니다.

이 새로운 시스템(`dtype_conversion_mode="all"` 사용)은 연관적이고 교환 가능하며, 최종적으로 어떤 너비의 부동 소수를 사용할지 쉽게 제어할 수 있습니다(더 넓은 부동 소수로 자동 변환되지는 않음). 오버플로우와 정밀도 손실의 위험이 있지만 `dtype_conversion_mode="safe"`을 사용하면 이러한 경우를 명시적으로 처리할 수 있습니다. 이 두 가지 모드는 [다음 섹션](#two_modes)에서 더 자세히 설명됩니다.

In [ ]:
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="all")

<a name="two_modes">
</a>

## 두 가지 모드: ALL 모드와 SAFE 모드

새로운 형식 승격 시스템에서는 `ALL` 모드와 `SAFE` 모드의 두 가지 모드를 도입합니다. `SAFE` 모드는 정밀도 손실 또는 비트 확대를 초래할 수 있는 '위험한' 승격에 대한 우려를 완화하는 데 사용됩니다.

### Dtypes

간결하게 표현하기 위해 다음 약어를 사용하겠습니다.

- `b`는 `tf.bool`을 의미합니다.
- `u8`은 `tf.uint8`을 의미합니다.
- `i16`은 `tf.int16`을 의미합니다.
- `i32`는 `tf.int32`를 의미합니다.
- `bf16`은 `tf.bfloat16`을 의미합니다.
- `f32`는 `tf.float32`를 의미합니다.
- `f64`는 `tf.float64`를 의미합니다.
- `i32*`는 Python `int` 또는 약한 형식의 `i32`를 의미합니다.
- `f32*`는 Python `float` 또는 약한 형식의 `f32`를 의미합니다.
- `c128*`은 Python `complex` 또는 약한 형식의 `c128`을 의미합니다.

별표(*)는 해당 유형이 "약한" 형식임을 나타냅니다. 이러한 형식은 시스템에 의해 일시적으로 추론되며 다른 형식으로 대체될 수 있습니다. 이 개념은 [여기](#weak_tensor)에서 더 자세히 설명됩니다.

### 정밀도 손실 연산의 예제

다음 예제에서 `i32` + `f32`는 `ALL` 모드에서는 허용되지만 `SAFE` 모드에서는 정밀도 손실의 위험 때문에 허용되지 않습니다.

In [ ]:
# i32 + f32 returns a f32 result in ALL mode.
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="all")
a = tf.constant(10, dtype = tf.int32)
b = tf.constant(5.0, dtype = tf.float32)
a + b  # <tf.Tensor: shape=(), dtype=float32, numpy=15.0>

In [ ]:
# This promotion is not allowed in SAFE mode.
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="safe")
a = tf.constant(10, dtype = tf.int32)
b = tf.constant(5.0, dtype = tf.float32)
try:
  a + b
except TypeError as e:
   print(f'{type(e)}: {e}')  # TypeError: explicitly specify the dtype or switch to ALL mode.

### 비트 확대 연산의 예제

다음 예제에서 `i8` + `u32`는 `ALL` 모드에서 허용되지만 `SAFE` 모드에서는 입력의 비트 수보다 더 많은 비트를 사용하는 비트 확대 때문에 허용되지 않습니다. 새로운 타입 승격 의미 체계는 필요한 비트 확대만 허용합니다.

In [ ]:
# i8 + u32 returns an i64 result in ALL mode.
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="all")
a = tf.constant(10, dtype = tf.int8)
b = tf.constant(5, dtype = tf.uint32)
a + b

In [ ]:
# This promotion is not allowed in SAFE mode.
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="safe")
a = tf.constant(10, dtype = tf.int8)
b = tf.constant(5, dtype = tf.uint32)
try:
  a + b
except TypeError as e:
   print(f'{type(e)}: {e}')  # TypeError: explicitly specify the dtype or switch to ALL mode.

## 격자 기반 시스템

### 형식 승격 격자

새 형식 승격 작동은 다음 형식 승격 격자를 통해 결정됩니다.

![Type Promotion Lattice](https://tensorflow.org/guide/images/new_type_promotion/type_promotion_lattice.png)

보다 구체적으로 말하면, 두 형식 사이에서 이루어지는 승격은 두 노드(노드 자체 포함)의 첫 번째 공통 하위 항목을 찾아야 결정됩니다.

예를 들어 위 다이어그램에서 `i8`과 `i32`의 첫 번째 공통 하위 항목은 `i32`입니다. 그 이유는 화살표 방향을 따라가면 두 노드가 `i32`에서 처음으로 교차하기 때문입니다.

다른 예제와 마찬가지로 `u64`와 `f16` 사이의 결과 승격 형식은 `f16`입니다.

<a name="promotion_table">
</a>

### 형식 승격 테이블

격자를 따라가면 아래와 같은 바이너리 승격 테이블이 생성됩니다.

**참고**: `SAFE` 모드에서는 강조 표시된 셀이 허용되지 않습니다. `ALL` 모드는 모든 경우를 허용합니다.

![Type Promotion Table](https://tensorflow.org/guide/images/new_type_promotion/type_promotion_table.png)

## 새 형식 승격의 장점

우리는 새로운 형식 승격을 위해 JAX와 유사한 격자 기반 시스템을 채택했으며 이 시스템의 장점은 다음과 같습니다.

<a name="lattice_system_design">
</a>

#### 격자 기반 시스템의 장점

먼저, 격자 기반 시스템을 사용하면 세 가지 매우 중요한 속성이 보장됩니다.

- 존재(Existence): 모든 형식의 조합에 대해 고유한 결과 승격 형식이 있습니다.
- 교환 가능성(Commutativity): `a + b = b + a`
- 연관성(Associativity): `a + (b + c) = (a + b) = c`

이 세 가지 속성은 일관되고 예측 가능한 형식 승격 의미 체계를 구성하는 데 매우 중요합니다.

#### JAX와 유사한 격자 시스템의 장점

JAX와 유사한 격자 시스템의 또 다른 중요한 장점은 서명되지 않은 인트를 제외하고는 필요 이상의 넓은 승격을 피할 수 있다는 것입니다. 즉, 64비트 입력 없이는 64비트 결과를 얻을 수 없습니다. 기존 형식 승격에서 빈번하게 발생했던 불필요한 64비트 값을 피할 수 있기에 이러한 장점은 가속기 작업에 매우 유용합니다.

그러나 이 시스템에도 단점은 있습니다. 부동 소수/정수 혼합 승격으로 인해 정밀도 손실이 발생할 가능성이 매우 높습니다. 예를 들어, 아래 예제에서 `i64` + `f16`을 사용하면 `i64`가 `f16`으로 승격됩니다.

In [ ]:
# The first input is promoted to f16 in ALL mode.
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="all")
tf.constant(1, tf.int64) + tf.constant(3.2, tf.float16)  # <tf.Tensor: shape=(), dtype=float16, numpy=4.2>

이러한 우려를 해소하기 위해 우리는 이렇게 '위험한' 승격을 허용하지 않는 `SAFE` 모드를 도입했습니다.

**참고**: 격자 시스템을 구성할 때 고려해야 할 디자인 고려 사항에 대한 자세한 내용은 [JAX용 형식 승격 의미 체계 디자인](https://jax.readthedocs.io/en/latest/jep/9407-type-promotion.html)을 참조하세요.

<a name="weak_tensor">
</a>

## WeakTensor

### 개요

*약한 텐서*는 [JAX](https://jax.readthedocs.io/en/latest/type_promotion.html#weakly-typed-values-in-jax)의 개념과 유사한 "약한 형식"의 텐서입니다.

`WeakTensor`의 dtype은 시스템에 의해 일시적으로 추론되며, 다른 dtype을 따를 수 있습니다. 이 개념은 Python 스칼라 리터럴과 같이 명시적으로 사용자가 지정한 형식이 없는 TF 값과 값 사이의 이진 연산 내에서 원치 않는 형식 승격을 방지하기 위해 새로운 형식 승격에 도입되었습니다.

예를 들어, 아래 예제에서 `tf.constant(1.2)`에는 특정 dtype이 없으므로 "약한" 형식으로 간주됩니다. 따라서 `tf.constant(1.2)`은 `tf.constant(3.1, tf.float16)`의 형식을 따라가며 그 결과로 `f16`을 출력합니다.

In [ ]:
tf.constant(1.2) + tf.constant(3.1, tf.float16)  # <tf.Tensor: shape=(), dtype=float16, numpy=4.3>

### WeakTensor 구성

WeakTensor는 dtype을 지정하지 않고 텐서를 생성할 때 생성됩니다. 텐서의 문자열 표현 끝에서 약한 속성을 확인하면 텐서가 "약한" 텐서인지 확인할 수 있습니다.

**첫 번째 사례**: 사용자가 특정한 dtype이 없는 입력으로 `tf.constant`을 호출한 경우.

In [ ]:
tf.constant(5)  # <tf.Tensor: shape=(), dtype=int32, numpy=5, weak=True>

In [ ]:
tf.constant([5.0, 10.0, 3])  # <tf.Tensor: shape=(3,), dtype=float32, numpy=array([ 5., 10.,  3.], dtype=float32), weak=True>

In [ ]:
# A normal Tensor is created when dtype arg is specified.
tf.constant(5, tf.int32)  # <tf.Tensor: shape=(), dtype=int32, numpy=5>

**두 번째 사례**: 사용자가 특정한 dtype이 없는 입력이 [WeakTensor 지원 API](#weak_tensor_apis)로 전달되는 경우.

In [ ]:
tf.math.abs([100.0, 4.0])  # <tf.Tensor: shape=(2,), dtype=float32, numpy=array([100., 4.], dtype=float32), weak=True>

##새 형식 승격 사용 효과

다음은 새 형식 승격을 사용한 결과로 발생하는 변경 사항의 대략적인 목록입니다.

- 보다 일관되고 예측 가능한 프로모션 결과.
- 비트 확대 위험의 감소.
- `tf.Tensor` 수학적 더블 언더 메서드는 새로운 형식 승격을 사용.
- `tf.constant`는 `WeakTensor` 반환  가능.
- `tf.constant`는 `dtype` 인수와 다른 dtype을 가진 텐서 입력이 전달될 때 암시적 변환을 허용.
- `tf.Variable` 인플레이스 연산(`assign`, `assign-add`, `assign-sub`)의 경우 암시적 변환 허용.
- `tnp.array(1)` 및 `tnp.array(1.0)`은 32비트 WeakTensor를 반환.
- `WeakTensor`가 생성되어 [WeakTensor 지원 단항 및 바이너리 API](#weak_tensor_apis)에 사용됨.


### 보다 일관되고 예측 가능한 승격 결과

[격자 기반 시스템](#lattice_system_design)을 사용하면 새로운 형식 승격으로 일관성 있고 예측 가능한 형식 승격 결과를 생성할 수 있습니다.

#### 기존 형식 승격

연산 순서를 변경하면 이전 형식 승격을 사용하는 일관되지 않은 결과가 나타납니다.

In [ ]:
# Setup
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="legacy")
a = np.array(1, dtype=np.int8)
b = tf.constant(1)
c = np.array(1, dtype=np.float16)

In [ ]:
# (a + b) + c throws an InvalidArgumentError.
try:
  tf.add(tf.add(a, b), c)
except tf.errors.InvalidArgumentError as e:
  print(f'{type(e)}: {e}')  # InvalidArgumentError

In [ ]:
# (b + a) + c returns an i32 result.
tf.add(tf.add(b, a), c)  # <tf.Tensor: shape=(), dtype=int32, numpy=3>

#### 새 형식 승격

새 형식 승격은 순서에 관계 없이 일관된 결과를 생성합니다.

In [ ]:
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="all")
a = np.array(1, dtype=np.int8)
b = tf.constant(1)
c = np.array(1, dtype=np.float16)

In [ ]:
# (a + b) + c returns a f16 result.
tf.add(tf.add(a, b), c)  # <tf.Tensor: shape=(), dtype=float16, numpy=3.0>

In [ ]:
# (b + a) + c also returns a f16 result.
tf.add(tf.add(b, a), c)  # <tf.Tensor: shape=(), dtype=float16, numpy=3.0>

### 비트 확대 위험의 감소

#### 기존 형식 승격

기존 형식 승격은 종종 64비트 결과를 생성했습니다.

In [ ]:
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="legacy")

In [ ]:
np.array(3.2, np.float16) + tf.constant(1, tf.int8) + tf.constant(50)  # <tf.Tensor: shape=(), dtype=float64, numpy=54.19921875>

#### 새 형식 승격

새 형식 승격은 필요로 하는 최소 비트 수의 결과를 반환합니다.

In [ ]:
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="all")

In [ ]:
np.array(3.2, np.float16) + tf.constant(1, tf.int8) + tf.constant(50)  # <tf.Tensor: shape=(), dtype=float16, numpy=54.2>

### tf.Tensor 수학적 더블 언더 메서드

모든 `tf.Tensor` 수학적 더블 언더 메서드는 새로운 형식 승격을 사용합니다.

In [ ]:
-tf.constant(5)  # <tf.Tensor: shape=(), dtype=int32, numpy=-5, weak=True>

In [ ]:
tf.constant(5, tf.int16) - tf.constant(1, tf.float32)  # <tf.Tensor: shape=(), dtype=float32, numpy=4.0>

### tf.Variable 인플레이스 연산

`tf.Variable` 인플레이스 연산에서는 암시적 변환이 허용됩니다.

**참고**: 변수의 원래 dtype과 다른 dtype을 초래하는 모든 승격은 허용되지 않습니다. 이는 `tf.Variable`이 dtype을 변경할 수 없기 때문입니다.

In [ ]:
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="all")
a = tf.Variable(10, tf.int32)
a.assign_add(tf.constant(5, tf.int16))  # <tf.Variable shape=() dtype=int32, numpy=15>

### tf.constant 암시적 변환

이전 형식 승격에서 `tf.constant`의 입력 텐서는 dtype 인수와 동일한 dtype을 가져야 했습니다. 그러나 새로운 형식 승격에서는 암시적으로 텐서를 지정된 dtype으로 변환합니다.

In [ ]:
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="all")
a = tf.constant(10, tf.int16)
tf.constant(a, tf.float32)  # <tf.Tensor: shape=(), dtype=float32, numpy=10.0>

### TF-NumPy 배열

새 형식 승격을 사용하는 Python 입력의 경우 `tnp.array`의 기본값은 `i32*` 및 `f32*`입니다.

In [ ]:
tnp.array(1)  # <tf.Tensor: shape=(), dtype=int32, numpy=1, weak=True>

In [ ]:
tnp.array(1.0)  # <tf.Tensor: shape=(), dtype=int32, numpy=1, weak=True>

##입력 형식 추론

새 형식 승격에서 다양한 입력 형식이 추론되는 방식입니다.

- `tf.Tensor`: `tf.Tensor`에는 dtype 속성이 있으므로 더 이상 추론을 수행하지 않습니다.
- NumPy 형식: 여기에는 `np.array(1)`, `np.int16(1)`, `np.float`와 같은 형식이 포함됩니다. NumPy 입력에도 dtype 속성이 있으므로 결과 추론 형식으로 dtype 속성을 사용합니다. NumPy의 기본값은 `i64` 및 `f64`입니다.
- Python 스칼라/중첩 형식: 여기에는 `1`, `[1, 2, 3]`, `(1.0, 2.0)`와 같은 형식이 포함됩니다.
    - Python `int`는 `i32*`로 추론됩니다.
    - Python `float`은 `f32*`로 추론됩니다.
    - Python `complex`는 `c128*`로 추론됩니다.
- 입력이 위 범주에 속하지 않지만 dtype 속성이 있는 경우 dtype 속성을 결과 추론 형식으로 사용합니다.

# 추가 자료

새 형식 승격은 JAX-NumPy의 형식 승겨과 매우 유사합니다. 새 형식 승격과 디자인 선택에 대한 자세한 내용을 알고 싶다면 아래 리소스를 확인해 주세요.

- [JAX 형식 승격 의미 체계](https://jax.readthedocs.io/en/latest/type_promotion.html)
- [JAX용 형식 승격 의미 체계 디자인](https://jax.readthedocs.io/en/latest/jep/9407-type-promotion.html)
- [기존 TF-NumPy 승격 의미 체계](https://www.tensorflow.org/guide/tf_numpy#type_promotion)


# 참고 자료

<a name="weak_tensor_apis">
</a>

## WeakTensor 지원 API

아래는 `WeakTensor`를 지원하는 API 목록입니다.

단항 연산의 경우 사용자가 지정한 형식이 없는 입력이 전달되면 `WeakTensor`를 반환함을 의미합니다.

바이너리 연산은 [이곳](#promotion_table)의 승격 테이블 를 따릅니다. 두 입력의 승격 결과에 따라 `WeakTensor`를 반환할 수도 있고 반환하지 않을 수도 있습니다.

**참고**: 모든 계산 연산(`+`, `-`, `*`, ...)이 지원됩니다.

- `tf.bitwise.invert`
- `tf.clip_by_value`
- `tf.debugging.check_numerics`
- `tf.expand_dims`
- `tf.identity`
- `tf.image.adjust_brightness`
- `tf.image.adjust_gamma`
- `tf.image.extract_patches`
- `tf.image.random_brightness`
- `tf.image.stateless_random_brightness`
- `tf.linalg.diag`
- `tf.linalg.diag_part`
- `tf.linalg.matmul`
- `tf.linalg.matrix_transpose`
- `tf.linalg.tensor_diag_part`
- `tf.linalg.trace`
- `tf.math.abs`
- `tf.math.acos`
- `tf.math.acosh`
- `tf.math.add`
- `tf.math.angle`
- `tf.math.asin`
- `tf.math.asinh`
- `tf.math.atan`
- `tf.math.atanh`
- `tf.math.ceil`
- `tf.math.conj`
- `tf.math.cos`
- `tf.math.cosh`
- `tf.math.digamma`
- `tf.math.divide_no_nan`
- `tf.math.divide`
- `tf.math.erf`
- `tf.math.erfc`
- `tf.math.erfcinv`
- `tf.math.erfinv`
- `tf.math.exp`
- `tf.math.expm1`
- `tf.math.floor`
- `tf.math.floordiv`
- `tf.math.floormod`
- `tf.math.imag`
- `tf.math.lgamma`
- `tf.math.log1p`
- `tf.math.log_sigmoid`
- `tf.math.log`
- `tf.math.multiply_no_nan`
- `tf.math.multiply`
- `tf.math.ndtri`
- `tf.math.negative`
- `tf.math.pow`
- `tf.math.real`
- `tf.math.real`
- `tf.math.reciprocal_no_nan`
- `tf.math.reciprocal`
- `tf.math.reduce_euclidean_norm`
- `tf.math.reduce_logsumexp`
- `tf.math.reduce_max`
- `tf.math.reduce_mean`
- `tf.math.reduce_min`
- `tf.math.reduce_prod`
- `tf.math.reduce_std`
- `tf.math.reduce_sum`
- `tf.math.reduce_variance`
- `tf.math.rint`
- `tf.math.round`
- `tf.math.rsqrt`
- `tf.math.scalar_mul`
- `tf.math.sigmoid`
- `tf.math.sign`
- `tf.math.sin`
- `tf.math.sinh`
- `tf.math.softplus`
- `tf.math.special.bessel_i0`
- `tf.math.special.bessel_i0e`
- `tf.math.special.bessel_i1`
- `tf.math.special.bessel_i1e`
- `tf.math.special.bessel_j0`
- `tf.math.special.bessel_j1`
- `tf.math.special.bessel_k0`
- `tf.math.special.bessel_k0e`
- `tf.math.special.bessel_k1`
- `tf.math.special.bessel_k1e`
- `tf.math.special.bessel_y0`
- `tf.math.special.bessel_y1`
- `tf.math.special.dawsn`
- `tf.math.special.expint`
- `tf.math.special.fresnel_cos`
- `tf.math.special.fresnel_sin`
- `tf.math.special.spence`
- `tf.math.sqrt`
- `tf.math.square`
- `tf.math.subtract`
- `tf.math.tan`
- `tf.math.tanh`
- `tf.nn.depth_to_space`
- `tf.nn.elu`
- `tf.nn.gelu`
- `tf.nn.leaky_relu`
- `tf.nn.log_softmax`
- `tf.nn.relu6`
- `tf.nn.relu`
- `tf.nn.selu`
- `tf.nn.softsign`
- `tf.nn.space_to_depth`
- `tf.nn.swish`
- `tf.ones_like`
- `tf.realdiv`
- `tf.reshape`
- `tf.squeeze`
- `tf.stop_gradient`
- `tf.transpose`
- `tf.truncatediv`
- `tf.truncatemod`
- `tf.zeros_like`
- `tf.experimental.numpy.abs`
- `tf.experimental.numpy.absolute`
- `tf.experimental.numpy.amax`
- `tf.experimental.numpy.amin`
- `tf.experimental.numpy.angle`
- `tf.experimental.numpy.arange`
- `tf.experimental.numpy.arccos`
- `tf.experimental.numpy.arccosh`
- `tf.experimental.numpy.arcsin`
- `tf.experimental.numpy.arcsinh`
- `tf.experimental.numpy.arctan`
- `tf.experimental.numpy.arctanh`
- `tf.experimental.numpy.around`
- `tf.experimental.numpy.array`
- `tf.experimental.numpy.asanyarray`
- `tf.experimental.numpy.asarray`
- `tf.experimental.numpy.ascontiguousarray`
- `tf.experimental.numpy.average`
- `tf.experimental.numpy.bitwise_not`
- `tf.experimental.numpy.cbrt`
- `tf.experimental.numpy.ceil`
- `tf.experimental.numpy.conj`
- `tf.experimental.numpy.conjugate`
- `tf.experimental.numpy.copy`
- `tf.experimental.numpy.cos`
- `tf.experimental.numpy.cosh`
- `tf.experimental.numpy.cumprod`
- `tf.experimental.numpy.cumsum`
- `tf.experimental.numpy.deg2rad`
- `tf.experimental.numpy.diag`
- `tf.experimental.numpy.diagflat`
- `tf.experimental.numpy.diagonal`
- `tf.experimental.numpy.diff`
- `tf.experimental.numpy.empty_like`
- `tf.experimental.numpy.exp2`
- `tf.experimental.numpy.exp`
- `tf.experimental.numpy.expand_dims`
- `tf.experimental.numpy.expm1`
- `tf.experimental.numpy.fabs`
- `tf.experimental.numpy.fix`
- `tf.experimental.numpy.flatten`
- `tf.experimental.numpy.flip`
- `tf.experimental.numpy.fliplr`
- `tf.experimental.numpy.flipud`
- `tf.experimental.numpy.floor`
- `tf.experimental.numpy.full_like`
- `tf.experimental.numpy.imag`
- `tf.experimental.numpy.log10`
- `tf.experimental.numpy.log1p`
- `tf.experimental.numpy.log2`
- `tf.experimental.numpy.log`
- `tf.experimental.numpy.max`
- `tf.experimental.numpy.mean`
- `tf.experimental.numpy.min`
- `tf.experimental.numpy.moveaxis`
- `tf.experimental.numpy.nanmean`
- `tf.experimental.numpy.negative`
- `tf.experimental.numpy.ones_like`
- `tf.experimental.numpy.positive`
- `tf.experimental.numpy.prod`
- `tf.experimental.numpy.rad2deg`
- `tf.experimental.numpy.ravel`
- `tf.experimental.numpy.real`
- `tf.experimental.numpy.reciprocal`
- `tf.experimental.numpy.repeat`
- `tf.experimental.numpy.reshape`
- `tf.experimental.numpy.rot90`
- `tf.experimental.numpy.round`
- `tf.experimental.numpy.signbit`
- `tf.experimental.numpy.sin`
- `tf.experimental.numpy.sinc`
- `tf.experimental.numpy.sinh`
- `tf.experimental.numpy.sort`
- `tf.experimental.numpy.sqrt`
- `tf.experimental.numpy.square`
- `tf.experimental.numpy.squeeze`
- `tf.experimental.numpy.std`
- `tf.experimental.numpy.sum`
- `tf.experimental.numpy.swapaxes`
- `tf.experimental.numpy.tan`
- `tf.experimental.numpy.tanh`
- `tf.experimental.numpy.trace`
- `tf.experimental.numpy.transpose`
- `tf.experimental.numpy.triu`
- `tf.experimental.numpy.vander`
- `tf.experimental.numpy.var`
- `tf.experimental.numpy.zeros_like`